In [ ]:
import os
import glob
import shutil
import json
import tensorflow.keras as k
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras import backend as K
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications import InceptionResNetV2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import warnings
from tensorflow.keras.applications.nasnet import NASNetLarge
warnings.filterwarnings("ignore")
# !pip install -U efficientnet
# import efficientnet.tfkeras as efn
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
# cwd = os.getcwd()
# os.chdir('../input/bitempered-logistic-loss-direct-upload/')
# from tf_bi_tempered_loss import BiTemperedLogisticLoss
# os.chdir(cwd)

In [ ]:
data_path = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/'
data = pd.read_csv(data_path + 'train.csv')
data.head()

In [ ]:
data['Class'].value_counts().plot.pie(figsize=(5,5))
plt.show()

In [ ]:
BATCH_SIZE = 32
EPOCHS = 10
n_CLASS = 6
IMG_SIZE = 150
size = (IMG_SIZE,IMG_SIZE)
train_path = data_path + 'train/'
test_path = data_path + 'test/'

In [ ]:
train,val = train_test_split(data, test_size = 0.1, random_state = 7, stratify = data['Class'])
print(f"The train size : {train.shape} \nThe Validation size : {val.shape}")

In [ ]:
# ADD rescale=1./255 AFTERWARDS
# prep_func = tf.keras.applications.resnet50.preprocess_input

datagen_train = ImageDataGenerator(
#                     preprocessing_function = prep_func,
                    rescale=1./255,
                    rotation_range = 40,
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = True,
                    fill_mode = 'nearest')

datagen_val = ImageDataGenerator(
                        rescale=1./255,
#                         preprocessing_function = prep_func,
                        )

train_set = datagen_train.flow_from_dataframe(train,
                                 directory = train_path,
                                 seed=7,
                                 x_col = 'Image',
                                 y_col = 'Class',
                                 target_size = size,
                                 class_mode = 'categorical',
#                                  interpolation = 'nearest',
                                 shuffle = True,
                                 batch_size = int(BATCH_SIZE))

val_set = datagen_val.flow_from_dataframe(val,
                                 directory = train_path,
                                 seed=7,
                                 x_col = 'Image',
                                 y_col = 'Class',
                                 target_size = size,
                                 #color_mode="rgb",
                                 class_mode = 'categorical',
#                                  interpolation = 'nearest',
                                 shuffle = True,
                                 batch_size = BATCH_SIZE)

In [ ]:
def create_model_efficientnetb0():

        model = Sequential()
        
        e = efn.EfficientNetB0(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = 'imagenet')
        for l in e.layers[:-5]:
            l.trainable = False
        model.add(e)
        model.add(GlobalAveragePooling2D())
#         model.add(Flatten())
#         model.add(Dense(8, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
#         model.add(Dropout(0.5))
        model.add(Dense(n_CLASS, activation = 'softmax',dtype='float32'))

        return model

In [ ]:
def create_model_efficientnetb4():

        model = Sequential()
        
        e = efn.EfficientNetB4(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = 'imagenet')
        for l in e.layers[:-5]:
            l.trainable = False
        model.add(e)
        model.add(GlobalAveragePooling2D())
#         model.add(Flatten())
#         model.add(Dense(8, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
#         model.add(Dropout(0.5))
        model.add(Dense(n_CLASS, activation = 'softmax',dtype='float32'))

        return model

In [ ]:
def create_model_resnet50():

        model = Sequential()
        
        e = ResNet50(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = 'imagenet',pooling='avg')
#         for l in e.layers:
#             l.trainable = False
        e.layers[0].trainable = False
        model.add(e)
#         model.add(GlobalAveragePooling2D())
#         model.add(Flatten())
#         model.add(Dense(8, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
#         model.add(Dropout(0.5))
        model.add(Dense(n_CLASS, activation = 'softmax',dtype='float32'))

        return model

In [ ]:
def create_model_inceptionresnet():

        model = Sequential()
        
        e = InceptionResNetV2(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = 'imagenet')
        for layer in e.layers[:-14]:
    #print(l)
            layer.trainable = False
        model.add(e)

        return model

In [ ]:
def create_model_nasnetL():

        model = Sequential()
        
        e = NASNetLarge(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = 'imagenet')
        for layer in e.layers[:-14]:
    #print(l)
            layer.trainable = False
        model.add(e)

        return model

In [ ]:
def create_model_vgg16():

        model = Sequential()
        
        e = VGG16(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = 'imagenet')
        for layer in e.layers[:-14]:
    #print(l)
            layer.trainable = False
        model.add(e)

        return model

In [ ]:
def create_model_resnet152v2():

        model = Sequential()
        
        e = ResNet152V2(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = 'imagenet')
        for layer in e.layers[:-14]:
    #print(l)
            layer.trainable = False
        model.add(e)

        return model

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy(from_logits = False,
                                                   label_smoothing=0.0001,
                                                   name='categorical_crossentropy' )
es = EarlyStopping(monitor='val_loss', mode='min', patience=3,
                   restore_best_weights=True, verbose=1)
checkpoint_cb = ModelCheckpoint("INCEPTIONRESNETV2.h5",
                                save_best_only=True,
                                monitor = 'val_loss',
                                mode='min')
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 1,
                              min_lr = 0.0001,
                              mode = 'min',
                              verbose = 1)

In [ ]:
e = create_model_inceptionresnet()
model = Sequential()
model.add(e)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu'))
model.add(Dense(6,activation='softmax',dtype='float32'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = Adam(learning_rate = 1e-3),
                    loss = loss, #'categorical_crossentropy'
                    metrics = ['accuracy'])
history = model.fit(x = train_set,
                         validation_data = val_set,
                         epochs= EPOCHS,
                         batch_size = BATCH_SIZE,
                         #class_weight = d_class_weights,
#                          steps_per_epoch = STEP_SIZE_TRAIN,
#                          validation_steps = STEP_SIZE_VALID,
                         callbacks=[es, checkpoint_cb, reduce_lr]
                   )

In [ ]:
classes = (train_set.class_indices)
classes = dict((v,k) for k,v in classes.items())

In [ ]:
best_model = k.models.load_model('INCEPTIONRESNETV2.h5')

In [ ]:
predictions = []
name = []
s=0
# classes = list(data['Class'].unique())
# single_test_path = data_path + 'single_img/'
test_images = os.listdir(test_path)
for image in test_images:
    img = Image.open(test_path + image)
    img = img.convert('RGB')
    img = img.resize(size)
    img = img_to_array(img)/255
    img = np.expand_dims(img, axis=0)
    pred  = best_model.predict(img)
    predictions.append(classes[np.argmax(pred[0])])
    name.append(image)
    s+=1
    if s%100==0:
        print(f"{s} Images' Prediction Done")
    

sub = pd.DataFrame({'Image': test_images, 'Class': predictions})
sub.to_csv('submission.csv', index = False)

## Fine- tuning

In [ ]:
for i in best_model.layers:
    i.trainable = True
    
checkpoint_cb = ModelCheckpoint("INCEPTIONRESNETV2_ft.h5",
                                save_best_only=True,
                                monitor = 'val_loss',
                                mode='min')
best_model.compile(optimizer = Adam(learning_rate = 1e-4),
                    loss = loss, #'categorical_crossentropy'
                    metrics = ['accuracy'])
history = best_model.fit(x = train_set,
                         validation_data = val_set,
                         epochs= EPOCHS,
                         batch_size = BATCH_SIZE,
                         #class_weight = d_class_weights,
#                          steps_per_epoch = STEP_SIZE_TRAIN,
#                          validation_steps = STEP_SIZE_VALID,
                         callbacks=[es, checkpoint_cb, reduce_lr]
                   )